# K-means

In [79]:
import random
import numpy as np
# code source from http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html
from sklearn.metrics.pairwise import euclidean_distances

def initiate_centroids(data_array, k):
    #initiate an array for the K indexes of picking the intitial K centroids 
    initial_indexes = random.sample(range(len(data_array)), k)
    # return the array of data correspondoing to the k indexes
    K_centroids = np.array(data_array)[initial_indexes]
    
    return K_centroids

#def 

def K_means(data_array, k, iteration):
    
    # record the dimentions of the data array.
    dimens = len(data_array[1])
    
    # initiate an array of K centroids 
    initial_cens = initiate_centroids(data_array, k)
    
    #initiate an list storing the new cluster for each data point (e.g.,row in the matrix)
    new_clusters = [0]*len(data_array)
    
    while iteration>0:
        
        iteration -= 1
        
        ##### E Step
        # calculate the similarity matrix 
        simi_matrix = euclidean_distances(data_array, initial_cens)
        
        # return the indexes of the minimum number in each row in the matrix, which is 
        # the culster lable that data point belong to
        new_clusters = np.argmin(simi_matrix, axis=1)
        
        
        old_cens = initial_cens
        
        # reset the initial k centroids and be prepared to stored the new ones
        initial_cens = []
        #initate the centroids with same dimensions while each dimens is 0
        for i in range(k):
            initial_cens.append(np.array([float(0)]*dimens))
            
        initial_cens = np.array(initial_cens)
        
        #define a list for recording each cluster's number
        count = [0]*k
        
        # here is to loop each data point and aggregate the data's each feature colum(dimension)
        # into a certain cluster, after that, initial_cens[k] is the aggretated location of each
        # cluster
        
        ####### M Step
        for j in range(len(data_array)):
            # here initial_cens[new_clusters[j]] is to record the array that going to  add
            # each dimension of each data point that belong to the jth new cluster 
            initial_cens[new_clusters[j]]+=np.array(data_array[j])
            
            # to add the number of data points in each cluster, here new_clusters[j] is the index
            # of new cluster each data point belong to, thus count[new_clusters[j]] record 
            # the number of each cluster
            count[new_clusters[j]] += 1
            
            #here is to get the mean location for the k culters,  namely, initial_cens[k] become 
            #the new culster's location
        for t in range(k):
            #get the mean
            initial_cens[t] = initial_cens[t]/count[t]
            
            
    
    return initial_cens, new_clusters
            
    
        

# Purity and Gini

In [81]:
# learn code from this source: https://docs.python.org/2/library/collections.html#collections.Counter
# A Counter is a dict subclass for counting hashable objects. 
from collections import Counter

def compute_purity(data_array, cens, clusters, real_label):
    # Define a dictionary for storing the key(cluster K) and value(the collection
    # of the real lables of the points c in this cluster)
    cluster_lable_dic = {}
    
    ## here we will gather the real lables of the points in a certain cluster, 
    ## format: cluster K : (K_real, K_real , K_real, J_real, B_real)
    
    # here we loop from each data point
    for i in range(len(data_array)):
        # clusters[i] return the cluster index of the predicted cluster for data point i 
        # if the cluster index (e.g., predicted label) been recorded, just add the point i's read-label 
        if clusters[i] in cluster_lable_dic.keys():
            cluster_lable_dic[clusters[i]].append(real_label[i])
        #else if the cluster index (e.g., predicted label) have't been recorded before,add here
        else:
            cluster_lable_dic[clusters[i]]= [real_label[i]]
    
    #iniitate total count as 0
    count_total = 0
    
    for j in cluster_lable_dic.keys():
        count_total += Counter(cluster_lable_dic[j]).most_common()[0][1] #return the number of the most common element
    
    purity = float(count_total)/float(len(data_array))
    
    return purity
    

def compute_gini(data_array, cens, clusters, real_label):
    # Define a dictionary for storing the key(cluster K) and value(the collection
    # of the real lables of the points c in this cluster)
    cluster_lable_dic = {}
    
    ## here we will gather the real lables of the points in a certain cluster, 
    ## format: cluster K : (K_real, K_real , K_real, J_real, B_real)
    
    # here we loop from each data point
    for i in range(len(data_array)):
        # clusters[i] return the cluster index of the predicted cluster for data point i 
        # if the cluster index (e.g., predicted label) been recorded, just add the point i's read-label 
        if clusters[i] in cluster_lable_dic.keys():
            cluster_lable_dic[clusters[i]].append(real_label[i])
        #else if the cluster index (e.g., predicted label) have't been recorded before,add here
        else:
            cluster_lable_dic[clusters[i]]= [real_label[i]]
    
    # for each cluster, gini index =  aggregating the P(k)(1-P(k)), and P(k)= (num_in_k_cluster)/N
    # thus gini = (num_in_k_cluster)*(N- (num_in_k_cluster))/ (N**2)
    gini= 0
    
    for j in cluster_lable_dic.keys():
        #return a list, e.g., for cluster K which there are 10 point in it: [(K,5), (J,3), (A,2)]
        list_lable = Counter(cluster_lable_dic[j]).most_common()
        # total number of data point in cluster j
        total_num = len(cluster_lable_dic[j])
        #total categories(cluster) appear in cluster j
        total_cate = len(list_lable)
        
        for k in range(total_cate):
            #list_lable[k][1] is the number of the lable(cluster) denoted as "list_lable[k][1] "
            gini += float(list_lable[k][1]*(total_num - list_lable[k][1]))/float(total_num**2)
    
    gini = float(gini)/float(len(cluster_lable_dic.keys()))
    
    return gini
        
    

    

# MNIST 

In [96]:
import numpy as np
# source: http://scikit-learn.org/stable/datasets/index.html

from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST original')

#convert the MNIST dataset into array
mnist_data = np.array(mnist['data'])

cen_mnist_5, cluster_mnist_5 = K_means(mnist_data, 5, 50)
cen_mnist_10, cluster_mnist_10 = K_means(mnist_data, 10, 50)
cen_mnist_20, cluster_mnist_20 = K_means(mnist_data, 20, 50)


purity_5 = compute_purity(mnist_data, cen_mnist_5, cluster_mnist_5, mnist.target)
gini_5 = compute_gini(mnist_data, cen_mnist_5, cluster_mnist_5, mnist.target)
purity_10 = compute_purity(mnist_data, cen_mnist_10, cluster_mnist_10, mnist.target)
gini_10 = compute_gini(mnist_data, cen_mnist_10, cluster_mnist_10, mnist.target)
purity_20 = compute_purity(mnist_data, cen_mnist_20, cluster_mnist_20, mnist.target)
gini_20 = compute_gini(mnist_data, cen_mnist_20, cluster_mnist_20, mnist.target)

print("MNIST, K = 5, purity is {} and gini is {}".format(purity_5, gini_5))
print("MNIST, K = 10, purity is {} and gini is {}".format(purity_10, gini_10))
print("MNIST, K = 20, purity is {} and gini is {}".format(purity_20, gini_20))





MNIST, K = 5, purity is 0.45248571428571427 and gini is 0.5934476807517318
MNIST, K = 10, purity is 0.5851571428571428 and gini is 0.5206361175480044
MNIST, K = 20, purity is 0.7237 and gini is 0.37006016129846436


# 20NEWS

In [99]:
# code source from http://scikit-learn.org/stable/datasets/twenty_newsgroups.html#
# code source from http://www.nltk.org/book/ch02.html
# code source from http://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.euclidean_distances.html

from sklearn.datasets import fetch_20newsgroups

import nltk
from nltk.corpus import stopwords
import string


# extracts the archive contents in the ~/scikit_learn_data/20news_home folder 
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

################# this part is for pre-processing the text file which isn't mandatory
# get rid of the stopwords and punctuation
nltk.download('stopwords')
stop_word =stopwords.words('english')
punctuation = string.punctuation
stopw_punctuation = list(stop_word) + list(punctuation)

from nltk.tokenize import word_tokenize

# to tokenize the word
for j in range(len(newsgroups_train.data)):
    newsgroups_train.data[j] = " ".join([w for w in word_tokenize(newsgroups_train.data[j]) if w not in stopw_punctuation])
   # print(newsgroups_train.data[])
for j in range(len(newsgroups_test.data)):
    newsgroups_test.data[j] = " ".join([w for w in word_tokenize(newsgroups_train.data[j]) if w not in stopw_punctuation])

#print(len(newsgroups_train.data))
#print(newsgroups_train.data[1])

##############

# from http://scikit-learn.org/stable/datasets/twenty_newsgroups.html
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectors_train = vectorizer.fit_transform(newsgroups_train.data)
# the test data set don't need to fit
vectors_test = vectorizer.transform(newsgroups_test.data)
print(vectors_train.shape)
print(vectors_test.shape)

news_data = np.array(vectors_train.todense())


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/deshenghu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
(11314, 130088)
(7532, 130088)


In [105]:
cen_news_5, cluster_news_5 = K_means(news_data, 5, 100)
cen_news_10, cluster_news_10 = K_means(news_data, 10, 100)
cen_news_20, cluster_news_20 = K_means(news_data, 20, 100)

purity_5 = compute_purity(news_data, cen_news_5, cluster_news_5, newsgroups_train.target)
gini_5 = compute_gini(news_data, cen_news_5, cluster_news_5, newsgroups_train.target)
purity_10 = compute_purity(news_data, cen_news_10, cluster_news_10, newsgroups_train.target)
gini_10 = compute_gini(news_data, cen_news_10, cluster_news_10, newsgroups_train.target)
purity_20 = compute_purity(news_data, cen_news_20, cluster_news_20, newsgroups_train.target)
gini_20 = compute_gini(news_data, cen_news_20, cluster_news_20, newsgroups_train.target)

print("20NEWS, K = 5, purity is {} and gini is {}".format(purity_5, gini_5))
print("20NEWS, K = 10, purity is {} and gini is {}".format(purity_10, gini_10))
print("20NEWS, K = 20, purity is {} and gini is {}".format(purity_20, gini_20))

20NEWS, K = 5, purity is 0.17367862824818808 and gini is 0.8771791867317184
20NEWS, K = 10, purity is 0.2612692239703023 and gini is 0.7751934615060898
20NEWS, K = 20, purity is 0.3261445996111013 and gini is 0.6095694928431954


# Fashion

In [108]:
import csv

##The “mnist_train_fashion.csv” is downloaded from webpage of this link : https://www.kaggle.com/zalando-research/fashionmnist
## here we only use the training data set for running the program

# data_array_fashion is to store the training data
data_array_fashion = []
with open("/Users/deshenghu/Dropbox/Dataset_CS6220/mnist_train_fashion.csv") as t:
    #read csv file using csv.reader()
    #source : https://docs.python.org/3/library/csv.html
    reader = csv.reader(t)
    for row in reader:
        data_array_fashion.append(row)
        

# get the lables of the training data, which is the first element of each row
data_fashion_lable = [row[0] for row in data_array_fashion]

# delete the lable which is the first element of each row
for i in range(len(data_array_fashion)):
    del data_array_fashion[i][0]
    
data_fashion_tempt =  np.array(data_array_fashion)
# because each element ("number") in the array is str and thus need to be converted
data_fashion_training = [list(map(int, data_fashion_tempt[j])) for j in range(len(data_fashion_tempt))]

## after getting the training data and the corresponding lable, we need to run the k-means function to get the centroids and predicted cluster or centroides
cen_fashion_5, cluster_fashion_5 = K_means(data_fashion_training, 5, 50)
cen_fashion_10, cluster_fashion_10 = K_means(data_fashion_training, 10, 50)
cen_fashion_20, cluster_fashion_20 = K_means(data_fashion_training, 20, 50)


purity_5 = compute_purity(data_fashion_training, cen_fashion_5, cluster_fashion_5, data_fashion_lable)
gini_5 = compute_gini(data_fashion_training, cen_fashion_5, cluster_fashion_5, data_fashion_lable)
purity_10 = compute_purity(data_fashion_training, cen_fashion_10, cluster_fashion_10, data_fashion_lable)
gini_10 = compute_gini(data_fashion_training, cen_fashion_10, cluster_fashion_10, data_fashion_lable)
purity_20 = compute_purity(data_fashion_training, cen_fashion_20, cluster_fashion_20, data_fashion_lable)
gini_20 = compute_gini(data_fashion_training, cen_fashion_20, cluster_fashion_20, data_fashion_lable)

print("Fashion, K = 5, purity is {} and gini is {}".format(purity_5, gini_5))
print("Fashion, K = 10, purity is {} and gini is {}".format(purity_10, gini_10))
print("Fashion, K = 20, purity is {} and gini is {}".format(purity_20, gini_20))
    
    

Fashion, K = 5, purity is 0.45245 and gini is 0.5932141975998142
Fashion, K = 10, purity is 0.58425 and gini is 0.5048468011334925
Fashion, K = 20, purity is 0.7068333333333333 and gini is 0.37662570610810964
